In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from analysis_utils import wilcoxon_paried_test, highlight_max
from autorank import autorank, create_report
from IPython.display import display, Latex


In [2]:
pdidx = pd.IndexSlice

# Loading

### MLP & MO-MLP & SNN & MO-SNN & edRVFL

In [3]:
results_file = "./results/eval_stats.csv"
metrics_df = pd.read_csv(results_file, header=[0, 1], index_col=[0, 1])
metrics_df.columns = metrics_df.columns.remove_unused_levels()
metrics_df.columns.levels[0]

Index(['MLP', 'MO-MLP', 'MO-MLP (CE)', 'MO-MLP (Seq, Scl)',
       'MO-MLP (Seq, Vec)', 'MO-SNN', 'SNN', 'edAS', 'edAS (Acc.)',
       'edAS (CE)', 'edAS (Exp. Acc.)', 'edAS (Exp. Loss., W: bc)',
       'edAS (Exp. Loss., W: bcbc)', 'edAS (Loss)', 'edMLP (CE, Boost)',
       'edRVFL'],
      dtype='object', name='model')

In [4]:
test_df = metrics_df.loc[:, pdidx[:, "test_acc"]].swaplevel(0, 1).droplevel(1, 1) * 100
comparison_df = metrics_df.unstack(1).mean(0)[:, "test_acc"].unstack(0) * 100

# Analysis

## Accuracy Table

In [5]:
momlp_indices = [
       'edAS', 
       'edAS (Loss)',
       'edAS (Acc.)',
       'edAS (CE)',
       'edAS (Exp. Acc.)',

       'edRVFL', 
       'edMLP (CE, Boost)',

       'SNN', 
       'MO-SNN',

       'MO-MLP (CE)',
       'MLP', 
       'MO-MLP', 
       'MO-MLP (Seq, Scl)', 
       'MO-MLP (Seq, Vec)',
]
print(len(momlp_indices))

14


In [6]:
comparison_df = comparison_df[momlp_indices]
test_df = test_df[momlp_indices]

In [7]:
cd = test_df.unstack(0).mean(0).unstack(0)
s = cd.rank(1, ascending=False).mean(0).sort_values()
display(cd.sort_index()[s.index].style.apply(highlight_max, axis=1))
## Rank
print("Ranking")
print(s)
## Avg. Accuracies
print("Avg. Accuracies")
cd.mean(0).sort_values(ascending=False)

model,edAS (Exp. Acc.),edAS,edAS (Loss),edAS (Acc.),"MO-MLP (Seq, Vec)","MO-MLP (Seq, Scl)",MO-MLP,edAS (CE),edRVFL,"edMLP (CE, Boost)",MO-MLP (CE),MO-SNN,MLP,SNN
dataset,,,,,,,,,,,,,,
abalone,65.450192,65.694444,65.354406,65.292146,66.032088,66.202107,65.909962,65.340038,66.192529,65.007184,65.881226,66.106322,66.027299,65.134100
arrhythmia,71.216814,70.619469,70.464602,70.420354,71.460177,71.969027,71.703540,67.632743,70.862832,71.017699,71.924779,71.681416,69.402655,69.889381
bank,89.747788,89.736726,89.550885,89.730088,90.070796,90.110619,89.495575,89.557522,89.774336,89.603982,89.818584,89.756637,89.725664,89.351770
breast-cancer-wisc-prog,78.469388,78.775510,79.030612,79.183673,79.183673,76.581633,79.438776,76.326531,81.836735,78.061224,78.520408,78.469388,77.500000,76.530612
breast-tissue,75.909091,73.977273,74.090909,76.590909,75.909091,73.977273,71.931818,71.250000,74.659091,74.545455,72.727273,66.250000,69.090909,71.250000
cardiotocography-10clases,84.274953,84.486817,84.110169,83.903013,85.225989,85.197740,84.915254,83.978343,82.354049,84.110169,83.865348,83.629944,84.750471,83.822976
cardiotocography-3clases,92.956685,92.490584,93.079096,91.986817,92.660075,92.575330,92.933145,92.410546,92.175141,91.958569,91.774953,92.321092,92.678908,92.669492
chess-krvkp,99.446183,99.430538,99.352315,99.396120,99.358573,99.336671,99.317897,99.455569,97.640801,99.305382,99.220901,99.220901,99.311640,99.280350
congressional-voting,58.990826,60.160550,60.206422,60.160550,59.266055,59.220183,59.885321,59.633028,60.022936,59.426606,58.853211,59.816514,58.555046,59.587156


Ranking
model
edAS (Exp. Acc.)      5.075758
edAS                  5.439394
edAS (Loss)           5.878788
edAS (Acc.)           5.984848
MO-MLP (Seq, Vec)     6.060606
MO-MLP (Seq, Scl)     6.075758
MO-MLP                7.742424
edAS (CE)             7.803030
edRVFL                8.151515
edMLP (CE, Boost)     8.500000
MO-MLP (CE)           9.212121
MO-SNN                9.318182
MLP                   9.515152
SNN                  10.242424
dtype: float64
Avg. Accuracies


model
edAS                 86.173076
edAS (Acc.)          86.037957
edAS (Exp. Acc.)     85.988567
edAS (Loss)          85.846498
MO-MLP (Seq, Vec)    85.786112
MO-MLP (Seq, Scl)    85.654266
edMLP (CE, Boost)    85.390744
edRVFL               85.327864
MO-MLP               85.268593
edAS (CE)            85.244887
MO-MLP (CE)          85.186764
MO-SNN               84.715589
MLP                  84.469064
SNN                  83.262681
dtype: float64

## Wilcoxon Test

In [8]:
wilcoxon_table , wilcoxon_p_values = wilcoxon_paried_test(test_df)
res = wilcoxon_table.copy().astype(object)
res["sum"] = res.sum(axis=1)
sort_idx = res.sort_values(by="sum" , ascending=False).index

res = res[sort_idx].loc[sort_idx]

for col in res.columns:
    for index in res.index:
        value = res.at[index, col]
        if value == 1.0:
            res.at[index, col] = '+'
        elif value == -1.0:
            res.at[index, col] = '-'
        elif value == 0.0:
            res.at[index, col] = ''
            
res

,edAS,edAS (Exp. Acc.),edAS (Acc.),"MO-MLP (Seq, Vec)",edAS (Loss),"MO-MLP (Seq, Scl)","edMLP (CE, Boost)",edAS (CE),MO-MLP,edRVFL,MO-MLP (CE),MLP,MO-SNN,SNN
edAS,,,+,+,+,+,+,+,+,+,+,+,+,+
edAS (Exp. Acc.),,,,+,+,+,+,+,+,+,+,+,+,+
edAS (Acc.),-,,,,+,+,+,+,+,+,+,+,+,+
"MO-MLP (Seq, Vec)",-,-,,,,,+,+,+,+,+,+,+,+
edAS (Loss),-,-,-,,,,+,+,+,+,+,+,+,+
"MO-MLP (Seq, Scl)",-,-,-,,,,+,+,+,+,+,+,+,+
"edMLP (CE, Boost)",-,-,-,-,-,-,,,,+,+,+,+,+
edAS (CE),-,-,-,-,-,-,,,,,+,+,+,+
MO-MLP,-,-,-,-,-,-,,,,,+,+,+,+
edRVFL,-,-,-,-,-,-,-,,,,,,+,+
